In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
# load dataset
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [7]:
# labels
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [8]:
# scaling
X_train_scaled= X_train / 255
X_test_scaled= X_test / 255

In [9]:
classes[y_train[3][0]]

'deer'

In [10]:
y_train_categorical = keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
y_test_categorical = keras.utils.to_categorical(y_test, num_classes=10, dtype='float32')

In [11]:
# custom NN model
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
# converting numpy array to tf array
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [14]:
type(train_tf_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [15]:
# tf MirroredStrategy 
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [16]:
# no of GPU if available or no of CPU
strategy.num_replicas_in_sync

1

In [17]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [18]:
train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [19]:
# runnintg distributed training
%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 64s 298ms/step - loss: 1.9785 - accuracy: 0.2985
Epoch 2/50
200/200 [==============================] - 63s 312ms/step - loss: 1.8163 - accuracy: 0.3652
Epoch 3/50
200/200 [==============================] - 60s 299ms/step - loss: 1.7541 - accuracy: 0.3889
Epoch 4/50
200/200 [==============================] - 61s 304ms/step - loss: 1.7110 - accuracy: 0.4040
Epoch 5/50
200/200 [==============================] - 61s 303ms/step - loss: 1.6768 - accuracy: 0.4153
Epoch 6/50
200/200 [==============================] - 61s 306ms/step - loss: 1.6478 - accuracy: 0.4267
Epoch 7/50
200/200 [==============================] - 61s 305ms/step - loss: 1.6225 - accuracy: 0.4362
Epoch 8/50
200/200 [==============================] - 60s 297ms/step - loss: 1.6003 - accuracy: 0.4434
Epoch 9/50
200/200 [==============================] - 58s 291ms/step - loss: 1.5804 - accuracy: 0.4512
Epoch 10/50
200/200 [==============================] - 58s 292ms/step - l

In [20]:
# Measure training time on a CPU
%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 60s 295ms/step - loss: 1.9772 - accuracy: 0.2999
Epoch 2/50
200/200 [==============================] - 59s 296ms/step - loss: 1.8133 - accuracy: 0.3644
Epoch 3/50
200/200 [==============================] - 59s 292ms/step - loss: 1.7520 - accuracy: 0.3880
Epoch 4/50
200/200 [==============================] - 59s 293ms/step - loss: 1.7092 - accuracy: 0.4036
Epoch 5/50
200/200 [==============================] - 58s 291ms/step - loss: 1.6751 - accuracy: 0.4161
Epoch 6/50
200/200 [==============================] - 58s 291ms/step - loss: 1.6463 - accuracy: 0.4269
Epoch 7/50
200/200 [==============================] - 58s 291ms/step - loss: 1.6212 - accuracy: 0.4367
Epoch 8/50
200/200 [==============================] - 58s 292ms/step - loss: 1.5989 - accuracy: 0.4443
Epoch 9/50
200/200 [==============================] - 58s 292ms/step - loss: 1.5788 - accuracy: 0.4513
Epoch 10/50
200/200 [==============================] - 58s 291ms/step - l